In [1]:
import os
import random
import json
import requests
from operator import itemgetter
from typing import Union, List
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain.callbacks.manager import CallbackManager
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec, ConfigurableField
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import JsonOutputToolsParser
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

#custom libraries that we will use later in the app
from common.utils import (
    SQLSearchAgent
)
from common.callbacks import StdOutCallbackHandler
from common.prompts import CUSTOM_CHATBOT_PROMPT 

from dotenv import load_dotenv
load_dotenv("credentials.env")

from IPython.display import Markdown, HTML, display 

def printmd(string):
    display(Markdown(string))

In [2]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

COMPLETION_TOKENS = 2000

# We can run the everything with GPT3.5, but try also GPT4 and see the difference in the quality of responses
# You will notice that GPT3.5 is not as reliable when using multiple sources.

# llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], 
#                       temperature=0, max_tokens=COMPLETION_TOKENS)

# Uncomment below if you want to see the answers streaming
llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], temperature=0, max_tokens=COMPLETION_TOKENS, streaming=True, callback_manager=cb_manager, api_version="2024-05-01-preview")
# llm = ChatOllama(model = "llama3.2:latest", max_tokens = 128000, temperature=0, callbacks=cb_manager, disable_streaming = False)

In [3]:
sql_search = SQLSearchAgent(llm=llm, k=10, callback_manager=cb_manager,
                name="sqlsearch",
                description="useful when the questions includes the term: sqlsearch",
                verbose=False)

# Multi Agent

In [4]:
tools = [sql_search]

In [5]:
agent = create_openai_tools_agent(llm, tools, CUSTOM_CHATBOT_PROMPT)

In [6]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [7]:
def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos

In [8]:
brain_agent_executor = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [9]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

# random_session_id = "session428" 
# ramdom_user_id = "user325"
config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}
print(random_session_id, ramdom_user_id)

session24 user800


# Sample Output

In [ ]:
def chat_with_agent():  
    while True:  
        question = input("Enter your question (or type 'exit' to quit): ")  
        if question.lower() == "exit":  
            print("Goodbye!")  
            break  
        response = brain_agent_executor.invoke({"question": question}, config=config)["output"]  
        printmd(response)  
  
# Run the chat function  
chat_with_agent()  

Enter your question (or type 'exit' to quit):  Hi


Good morning! Ready for a productive day ahead? Let's get started with your daily standup update! Are you ready to provide your update for today?

Good morning! Ready for a productive day ahead? Let's get started with your daily standup update! Are you ready to provide your update for today?

Enter your question (or type 'exit' to quit):  Yes I am ready!


Great! Let's structure your update into three parts: Accomplishments, Plans, and Blockers. I'll help draft each section based on your recent activities.

### Accomplishments
- Successfully merged the pull request for "Direct message functionality" and "Initial memory commit" into the main branch.
- Addressed and closed the issue regarding the lack of a descriptive ReadMe file.

### Plans
- Continue working on the database setup for the codebase, as noted in the recently opened issue.

### Blockers
- You mentioned getting blocked due to a lack of tools for date and time segregation. We might need to explore solutions or workarounds for this issue.

Does this draft capture everything correctly, or would you like to add or modify any details?

...

Certainly! Let's update the plans section to include the other open issues.

### Plans
- Continue working on the database setup for the codebase, as noted in the recently opened issue.
- Address the issue regarding the primitive code structure and unprofessional folder arrangements.

Does this updated draft meet your expectations, or is there anything else you'd like to adjust?

Certainly! Let's update the plans section to include the other open issues.

### Plans
- Continue working on the database setup for the codebase, as noted in the recently opened issue.
- Address the issue regarding the primitive code structure and unprofessional folder arrangements.

Does this updated draft meet your expectations, or is there anything else you'd like to adjust?

Enter your question (or type 'exit' to quit):  looks great


I'm glad to hear that! Here's the final draft of your standup update:

### Accomplishments
- Successfully merged the pull request for "Direct message functionality" and "Initial memory commit" into the main branch.
- Addressed and closed the issue regarding the lack of a descriptive ReadMe file.

### Plans
- Continue working on the database setup for the codebase, as noted in the recently opened issue.
- Address the issue regarding the primitive code structure and unprofessional folder arrangements.

### Blockers
- You mentioned getting blocked due to a lack of tools for date and time segregation. We might need to explore solutions or workarounds for this issue.

Are you ready to submit this update, or is there anything else you'd like to add or change?

I'm glad to hear that! Here's the final draft of your standup update:

### Accomplishments
- Successfully merged the pull request for "Direct message functionality" and "Initial memory commit" into the main branch.
- Addressed and closed the issue regarding the lack of a descriptive ReadMe file.

### Plans
- Continue working on the database setup for the codebase, as noted in the recently opened issue.
- Address the issue regarding the primitive code structure and unprofessional folder arrangements.

### Blockers
- You mentioned getting blocked due to a lack of tools for date and time segregation. We might need to explore solutions or workarounds for this issue.

Are you ready to submit this update, or is there anything else you'd like to add or change?

In [11]:
printmd(brain_agent_executor.invoke({"question": "Hi"}, 
                                    config=config)["output"])

Good morning! Ready for a productive day ahead? Let's get started with your daily standup update! Are you ready to provide your update for today?

Good morning! Ready for a productive day ahead? Let's get started with your daily standup update! Are you ready to provide your update for today?

In [12]:
printmd(brain_agent_executor.invoke({"question": "Give me a summary of the github issues from yesterday"}, 
                                    config=config)["output"])

Here's a summary of the GitHub issues from yesterday, January 8th, 2025, for the repository `ashishkj21/DailyPulseFastAPI`:

1. **Issue: Code Structure**
   - **Opened:** 2025-01-08 23:51:42
   - **URL:** [Issue #2](https://github.com/ashishkj21/DailyPulseFastAPI/issues/2)
   - **Description:** The code structure is described as very primitive, with folder structures not arranged in a professional manner.

2. **Issue: Github Repo Lacks ReadMe**
   - **Opened:** 2025-01-08 22:07:42
   - **URL:** [Issue #1](https://github.com/ashishkj21/DailyPulseFastAPI/issues/1)
   - **Description:** The repository lacks a descriptive readme file, making it difficult to understand the project requirements.

These issues highlight areas for improvement in documentation and project structure. Would you like to discuss plans to address these issues or any other specific details?

Here's a summary of the GitHub issues from yesterday, January 8th, 2025, for the repository `ashishkj21/DailyPulseFastAPI`:

1. **Issue: Code Structure**
   - **Opened:** 2025-01-08 23:51:42
   - **URL:** [Issue #2](https://github.com/ashishkj21/DailyPulseFastAPI/issues/2)
   - **Description:** The code structure is described as very primitive, with folder structures not arranged in a professional manner.

2. **Issue: Github Repo Lacks ReadMe**
   - **Opened:** 2025-01-08 22:07:42
   - **URL:** [Issue #1](https://github.com/ashishkj21/DailyPulseFastAPI/issues/1)
   - **Description:** The repository lacks a descriptive readme file, making it difficult to understand the project requirements.

These issues highlight areas for improvement in documentation and project structure. Would you like to discuss plans to address these issues or any other specific details?

In [ ]:
printmd(brain_agent_executor.invoke({"question": "Give me a summary of the github issues from 9th January"}, 
                                    config=config)["output"])

In [13]:
printmd(brain_agent_executor.invoke({"question": "sqlsearch Give me all details of draft"}, 
                                    config=config)["output"])

I'll need to use the `sqlsearch` tool to fetch the details of the draft from the database. Could you please specify which draft details you are looking for, or should I fetch all available draft details?

I'll need to use the `sqlsearch` tool to fetch the details of the draft from the database. Could you please specify which draft details you are looking for, or should I fetch all available draft details?